<a href="https://colab.research.google.com/github/p-tech/wbs-dm/blob/main/sqlite-exercise/Exercise_SQLite_DB_Joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEP 1: CREATE the SQLite database;**


We need to import the sqlite3 module and create the database and tables.  You'll see this follows the syntax we have used on previous weeks.


In [ ]:
import sqlite3

#This statement creates a connection labelled as conn.  This will be used throughout to ensure the consistency for when we start to query the database tables.
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE olist_customers (
    customer_id VARCHAR(32) PRIMARY KEY,
    customer_unique_id VARCHAR(32),
    customer_zip_code_prefix INT,
    customer_city VARCHAR(255),
    customer_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE olist_geolocation (
    geolocation_zip_code_prefix INT,
    geolocation_lat FLOAT,
    geolocation_lng FLOAT,
    geolocation_city VARCHAR(255),
    geolocation_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE olist_order_items (
    order_id VARCHAR(32),
    order_item_id INT,
    product_id VARCHAR(32),
    seller_id VARCHAR(32),
    shipping_limit_date DATETIME,
    price FLOAT,
    freight_value FLOAT,
    PRIMARY KEY (order_id, order_item_id)
);
''')

cursor.execute('''
CREATE TABLE olist_order_payments (
    order_id VARCHAR(32),
    payment_sequential INT,
    payment_type VARCHAR(50),
    payment_installments INT,
    payment_value FLOAT,
    PRIMARY KEY (order_id, payment_sequential)
);
''')

cursor.execute('''
CREATE TABLE olist_order_reviews (
    review_id VARCHAR(32) PRIMARY KEY,
    order_id VARCHAR(32),
    review_score INT,
    review_comment_title TEXT,
    review_comment_message TEXT,
    review_creation_date DATETIME,
    review_answer_timestamp DATETIME
);
''')

cursor.execute('''
CREATE TABLE olist_orders (
    order_id VARCHAR(32) PRIMARY KEY,
    customer_id VARCHAR(32),
    order_status VARCHAR(50),
    order_purchase_timestamp DATETIME,
    order_approved_at DATETIME,
    order_delivered_carrier_date DATETIME,
    order_delivered_customer_date DATETIME,
    order_estimated_delivery_date DATETIME
);
''')

cursor.execute('''
CREATE TABLE olist_products (
    product_id VARCHAR(32) PRIMARY KEY,
    product_category_name VARCHAR(255),
    product_name_lenght FLOAT,
    product_description_lenght FLOAT,
    product_photos_qty FLOAT,
    product_weight_g FLOAT,
    product_length_cm FLOAT,
    product_height_cm FLOAT,
    product_width_cm FLOAT
);
''')

cursor.execute('''
CREATE TABLE olist_sellers (
    seller_id VARCHAR(32) PRIMARY KEY,
    seller_zip_code_prefix INT,
    seller_city VARCHAR(255),
    seller_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE product_category_translation (
    product_category_name VARCHAR(255) PRIMARY KEY,
    product_category_name_english VARCHAR(255)
);
''')

#This saves the chnages to the databae.  Up unitl this point the executed SQL statement isn't stored, changes are not immediatley saved.
conn.commit()

print("Database and tables created successfully!")


Database and tables created successfully!


**STEP 2: Check Tables Created:**

Run the command to show the database tables created and the structure.

In [ ]:

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    print(f"Table: {table_name[0]}")
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  Column: {col[1]}, Type: {col[2]}, NotNull: {col[3]}, DefaultVal: {col[4]}, PrimaryKey: {col[5]}")
    print("-" * 20)




**STEP 3: Upload Files:**

Run this box multiple times to upload the relevant csv files. Or drag the files across to the Files window from your desktop.

In [ ]:


from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


**STEP 4: Load CSV files into the database tables:**

This will populate the database tables with the data from the csv files.  No need to write INSERT statements.

You need to make sure the correct files are loaded into the corresponding tables.

In [ ]:
import csv

def import_csv_to_table(csv_file, table_name):
    #opens the file aas read only 'r', doesn't allow the origianl csv to be changed.
    with open(csv_file, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row if present
        for row in csv_reader:
            #? creates a placeholder for each column in the CSV file. ['?','?','?'] - Join makes it a string so it can then be inserted.
            # use of the '?' reduce risk of SQL injection
            placeholders = ', '.join(['?' for _ in row])
            #Assumes that the CSV and table have the same structure (this could be an issue) Would have to specify column names if different.
            sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(sql, row)

# Import data from CSV files into the relevant table - Student_Table goes into student table.  teh import_csv_to_table is the function, passing the two values across.
try:
    import_csv_to_table('olist_customers_dataset.csv', 'olist_customers')
    import_csv_to_table('olist_geolocation_dataset.csv', 'olist_geolocation')
    import_csv_to_table('olist_order_items_dataset.csv', 'olist_order_items')
    import_csv_to_table('olist_order_payments_dataset.csv', 'olist_order_payments')
    import_csv_to_table('olist_orders_dataset.csv', 'olist_orders')
    import_csv_to_table('olist_products_dataset.csv', 'olist_products')
    import_csv_to_table('olist_sellers_dataset.csv', 'olist_sellers')
    import_csv_to_table('product_category_name_translation.csv', 'product_category_translation')
    conn.commit()
    print("Data imported successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()  # Rollback changes if an error occurred



**STEP 5: Check Data has loaded**

Query each database table and load the data into a dataframe and display the first 5 lines

**ONLY RUN IF YOU NEED TO DELETE THE DATA IN THE TABLES**

If you run go back to **STEP 4** and re-run from there.

In [ ]:
# only run if you need to reset the tables without deleting the databae and starting again - then re-run the box previous box.
# Delete all data from the tables
cursor.execute("PRAGMA foreign_keys = OFF")
cursor.execute("DELETE FROM olist_customers")
cursor.execute("DELETE FROM olist_geolocation")
cursor.execute("DELETE FROM olist_order_items")
cursor.execute("DELETE FROM olist_order_payments")
cursor.execute("DELETE FROM olist_order_reviews")
cursor.execute("DELETE FROM olist_orders")
cursor.execute("DELETE FROM olist_products")
cursor.execute("DELETE FROM olist_sellers")
cursor.execute("DELETE FROM product_category_translation")
cursor.execute("PRAGMA foreign_keys = ON")

# Commit the changes
conn.commit()

conn.commit()
print("Database Deleted - restart.")



Database Deleted - restart.


In [ ]:
import pandas as pd

customersOrdersValue_df = pd.read_sql_query("""
SELECT c.customer_id,
       COUNT(DISTINCT o.order_id) AS total_orders,
       SUM(oi.price) AS total_order_value
FROM olist_customers c
JOIN olist_orders o ON c.customer_id = o.customer_id
JOIN olist_order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id
ORDER BY total_order_value DESC
""", conn)

print(customersOrdersValue_df)
customersOrdersValue_df.to_csv('customerOrdersValue_df.csv', index=False)


In [ ]:
#show number and value of orders for each customer

customersOrdersValue_df = pd.read_sql_query("""

""", conn)



print(customersOrdersValue_df)
customersOrdersValue_df.to_csv('customerOrdersValue_df.csv', index=False)

In [ ]:
#where do the customers come from - location

cLocation_df = pd.read_sql_query("""

""", conn)
print(cLocation_df)
cLocation_df.to_csv('cLocation_df.csv', index=False)

In [ ]:
#look at which sellers are selling the most items

sellersSales_df = pd.read_sql_query("""

""", conn)
print(sellersSales_df)
sellersSales_df.to_csv('sellersSales_df.csv', index=False)

#AVG order value for the seller
cSellersAVG_df = pd.read_sql_query("""

""", conn)

print(cSellersAVG_df)
cSellersAVG_df.to_csv('cSellersAVG_df.csv', index=False)

In [ ]:
#take a customer that has the most orders and pull out all the individual order items - is there any consistency - are they buying form the same categories every time.
customersOrders_df = pd.read_sql_query("""

""", conn)

print(customersOrders_df.head(5))

#get customer items fomr order
customerItems_df = pd.read_sql_query("""

""", conn)
print(customerItems_df)

In [ ]:
#How are customers paying - which payment methods are used the most

paymentMethods_df = pd.read_sql_query("""

""", conn)
print(paymentMethods_df)
paymentMethods_df.to_csv('paymentMethods_df.csv', index=False)

In [ ]:
#if possible, look at the payment methods and location - Identify the area that spends the highest amount - you could look at hteh product categories as well.

paymentByLocation_df = pd.read_sql_query("""

""", conn)

# Print formatted output
print(paymentByLocation_df)
paymentByLocation_df.to_csv('paymentByLocation_df.csv', index=False)

In [ ]:
#Range of payment by location
paymentByLocationRange_df = pd.read_sql_query("""

""", conn)

# Round monetary values for better readability
paymentByLocationRange_df[['Total Spent', 'Max Payment', 'Min Payment', 'Payment Range']] = paymentByLocationRange_df[['Total Spent', 'Max Payment', 'Min Payment', 'Payment Range']].round(2)

# Print formatted output
paymentByLocationRange_df.to_csv('paymentByLocationRange_df.csv',index=False)


In [ ]:
#What's the average order spend in the categories - show the category name in English

avgOrderSpendByCategory_df = pd.read_sql_query("""

""", conn)

# Print formatted output
print(avgOrderSpendByCategory_df.to_string(index=False))
avgOrderSpendByCategory_df.to_csv('avgOrderSpendByCategory_df.csv', index=False)

In [ ]:
#get categories in English

avgOrderSpendByCategory_df = pd.read_sql_query("""

""", conn)

# Print formatted output
print(avgOrderSpendByCategory_df.to_string(index=False))